In [1]:
import pandas as pd

passed_data = []
data_procession = []
with open('/home/ouconstand/data/Virus_Host/Virion.csv','r', encoding='ISO-8859-1') as f:
    for line in f.read().splitlines():
        data = line.split('\t')
        if len(data) < 34:
            passed_data.append(data)
        else:
            if data[8] == '' or data[13] == '' or data[3] == '': # delete non_family information data
                pass
            else:
                data_procession.append(data)

data = pd.DataFrame(data_procession[1:-1],columns=data_procession[0])

In [2]:
data.shape

(478970, 34)

In [3]:
len(set(data['VirusTaxID']))

8853

In [4]:
with open('/home/ouconstand/wang_0207.txt','r') as f:
    wang = [i.strip('\n') for i in f.readlines()]

data = data[data['VirusFamily'].isin(wang)]
data.shape

(374206, 34)

In [5]:
len(set(data['VirusTaxID']))

7866

In [8]:
count_all = pd.DataFrame(columns=['VirusFamily','H_trans','H_non_trans','NH_trans','NH_non_trans','Infect_Multi_Family','Infect_One_Family','IMF_human','IOF_human','Total'])

virus_family = set(data['VirusFamily'])

n = 0
for one_family in virus_family:
    
    train_data = data[data['VirusFamily'] == one_family]
    virusID_hostfamily = train_data[['VirusTaxID','HostFamily']]
    virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
    virusID_hostfamily.reset_index(drop=True) # reset index

    dic = {}

    homo_total = 0
    non_homo_total = 0

    homo_trans = 0
    homo_non_trans = 0

    non_homo_trans = 0
    non_homo_non_trans = 0

    infect_many = 0
    infect_one = 0

    infect_many_human = 0
    infect_one_human = 0

    for i,j in zip(virusID_hostfamily['VirusTaxID'], virusID_hostfamily['HostFamily']):
        dic[i] = dic.get(i, []) + [j]

    for value in dic.values():
        if len(value) > 1: # infect many host family (>= 2)
            infect_many += 1
            if 'hominidae' in value: # the virus can infect human and it infects many host family
                homo_trans += 1
                homo_total += 1
                infect_many_human += 1
            else:
                non_homo_trans += 1
                non_homo_total += 1
            

        else: # just infect one host family
            infect_one += 1
            if 'hominidae' in value: # the virus just infects human
                homo_non_trans += 1
                homo_total += 1
                infect_one_human += 1
            else:
                non_homo_non_trans += 1
                non_homo_total += 1
            

        
    count_all.loc[n] = [one_family,homo_trans,homo_non_trans,non_homo_trans,non_homo_non_trans,infect_many,infect_one,infect_many_human,infect_one_human,homo_total+non_homo_total]
    n += 1



<ipython-input-8-ed52526f1d3f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-8-ed52526f1d3f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-8-ed52526f1d3f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [9]:
count_all.shape

(28, 10)

In [ ]:
count_all.to_csv('/mnt/c/Users/ouyangkang/Desktop/count_all_0208.csv',index=None)

In [7]:
a = ['1']
if '1' in a : print('hello')

hello


In [6]:
useful_info = data[['VirusTaxID','HostFamily','VirusFamily']]

In [48]:
id_family = useful_info[['VirusTaxID','VirusFamily']]
print(id_family.shape)
id_family.drop_duplicates(subset=['VirusTaxID','VirusFamily'],inplace=True)
print(id_family.shape)

(374206, 2)
(7866, 2)


<ipython-input-48-4b17eb87747f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_family.drop_duplicates(subset=['VirusTaxID','VirusFamily'],inplace=True)


In [49]:
virus_hostF = useful_info[['VirusTaxID','HostFamily']]
print(virus_hostF.shape)
virus_hostF.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True)
print(virus_hostF.shape)

(374206, 2)
(12159, 2)


<ipython-input-49-59244dc695a2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus_hostF.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True)


In [50]:
inject_f_count = virus_hostF.groupby("VirusTaxID").agg('count')
only_one = inject_f_count[inject_f_count["HostFamily"] == 1]
only_one = only_one.reset_index()
only_one.columns = ['VirusTaxID','HostFamily']
more_than_one = inject_f_count[inject_f_count["HostFamily"] > 1]
more_than_one = more_than_one.reset_index()
more_than_one.columns = ['VirusTaxID','HostFamily']

In [51]:
only_one.shape

(6931, 2)

In [38]:
only_one

,VirusTaxID,HostFamily
0,1000586,1
1,1000587,1
2,1000650,1
3,1001081,1
4,1001627,1
...,...,...
7802,996628,1
7803,996629,1
7804,996630,1
7805,998085,1


In [ ]:
set(more_than_one['VirusTaxID']) & set(only_one['VirusTaxID'])

In [52]:
more_than_one.shape

(935, 2)

In [ ]:
list(only_one["VirusTaxID"])

In [53]:
one_home = virus_hostF[virus_hostF['VirusTaxID'].isin(list(only_one['VirusTaxID']))]
one_home = one_home[one_home['HostFamily'] == 'hominidae']

In [54]:
one_home.shape

(1025, 2)

In [55]:
more = virus_hostF[virus_hostF['VirusTaxID'].isin(list(more_than_one['VirusTaxID']))]
more = more[more['HostFamily'] == 'hominidae']

In [56]:
more.shape

(427, 2)